$$\textbf{Optimización I. Tarea 6}$$
$$\textit{Y. Sarahi García Gozález}$$

$\color{purple}{Librerías \space }$ <br>

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import functions as fn #libreria con las funciones Himmelblau,Beale,Rosenbrock,Hartman
from math import exp

In [2]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version", python_version())
print("Numpy version", np.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version 3.11.7
Numpy version 1.26.3


In [3]:
#imprimimos el epsilon de la máquina
epsilon = np.finfo(float).eps
print("Epsilon de la máquina:", epsilon)

Epsilon de la máquina: 2.220446049250313e-16


In [4]:
#definicion de funcion para para las graficas dee los contornos

def contornosFnc2D(fncf, puntos, xleft, xright, ybottom, ytop, levels):
    # Crea una discretización uniforme del intervalo [xleft, xright]
    ax = np.linspace(xleft, xright, 250)
    # Crea una discretización uniforme del intervalo [ybottom, ytop]
    ay = np.linspace(ybottom, ytop, 200)
    # La matriz mX que tiene las abscisas 
    mX, mY = np.meshgrid(ax, ay)
    # Se crea el arreglo mZ con los valores de la función en cada nodo
    mZ = mX.copy()
    for i, y in enumerate(ay):
        for j, x in enumerate(ax):
            mZ[i,j] = fncf(np.array([x, y]))
    # Grafica de las curvas de nivel
    fig, ax = plt.subplots()
    CS = ax.contourf(mX, mY, mZ, levels, cmap='coolwarm')
    plt.colorbar(CS, ax=ax)
    # Grafica los puntos dados
    puntos_x = [p[0] for p in puntos]
    puntos_y = [p[1] for p in puntos]
    ax.plot(puntos_x, puntos_y, 'r-',label="Sucesión")
    ax.plot(puntos_x[0], puntos_y[0], 'g*',label="punto inicial")
    ax.plot(puntos_x[-1], puntos_y[-1], 'b*',label="mínimo encontrado")
     # Grafica los puntos como puntos rojos
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title('Contornos de la función sucesión')
    plt.legend()
    plt.show()

In [5]:
def imprime(f,D_f,metodo,metodo_args):

    '''
    Esta función imprime el número de iteraciones y el punto de convergencia del método de optimizacion 
    "metodo" que se aplica a la funcion f. Además imrpime f y la norma de su gradiente evaluadas en 
    el punto de convergencia de metodo.

    En el caso de que la funcion f sea de R2 a R, imprime sus contornos de nivel y la sucesion
    de puntos generada por metodo.
    '''

    xk, indicador, k,m = metodo(*metodo_args)

    print('It = ', k)
    print('x_k       = ', xk)
    print('f(x_k)    = ', f(xk))
    print('||gk||   = ', np.linalg.norm(D_f(xk)))


    if(indicador):
        print("Sí se cumplio el criterio de convergencia")
    if m is not None:
        # Usamos list comprehension para crear la lista de puntos
        sucesion = [[fila[0], fila[1]] for fila in m[:k]]

        contornosFnc2D(f, sucesion,xleft=-5.5, xright=5.5, ybottom=-5.5, ytop=5.5,
                   levels=[0.5, 5, 10, 25, 50, 100, 150, 250, 400])

In [6]:
def imprime_1(metodo,metodo_args):

    '''
    Esta función imprime:
    - la dimensión $n$,
    - el  número $k$ de iteraciones realizadas,
    - las primeras y últimas 4 entradas del punto $\mathbf{x}_k$ que devuelve el algoritmo,
    - la norma del r_k, 
    - la variable $bres$ para saber si el algoritmo puedo converger.

    '''

    n,xk,rk,k,indicador= metodo(*metodo_args)

    
    print('Dimensión de A = ', n )
    print('Número de iteraciones = ', k )
    print('Primeras cuatro entradas de x_k= ', xk[:4])
    print('Últimas cuatro entradas de x_k= ', xk[-4:])
    print('Norma residual ||rk||   = ', np.linalg.norm(rk))


    if(indicador):
        print("Sí se cumplio el criterio de convergencia")


In [70]:
def imprime_2(f,n,x0,metodo,metodo_args):

    '''
    Esta función imprime:
    - el último punto  $x_k$,
    - el gradiente evaluado en el último punto  $x_k$,
    - el  número $k$ de iteraciones realizadas,
    - número de veces que se aplica el reinicio
    - la variable $bres$ para saber si el algoritmo puedo converger.

    '''

    xk,gk,k,br,indicador= metodo(*metodo_args)
    print('Dimensión de x0, n = ',n)
    print('f(x_0) = ',f(x0))
    print('Número de iteraciones = ', k ) 
    print('f(x_k)= ', f(xk))
    if n==2:
        print('x_k= ', xk)
    else:
        print('Primeras cuatro entradas de x_k= ', xk[:4])
        print('Últimas cuatro entradas de x_k= ', xk[-4:])

    print('Norma del gradiente en x_k= ', np.linalg.norm(gk) )
    print('Contador br = ', br)


    if(indicador):
        print("Sí se cumplio el criterio de convergencia")

$\color{purple}{Ejercicio \space 1}$ <br>

In [10]:
def gradiente_conjugado_lineal(x_0,A,b,max_iter,tol):
    n=np.shape(A)[0]
    bres=False
    r_k=(A@x_0)-b
    p_k=-r_k
    x_k=x_0
    for k in range(max_iter):
        r=np.linalg.norm(r_k)
        if r<tol:
            bres=True
            return n,x_k,r_k,k,bres


        Ap=A@p_k

        alpha_k=(r**2)/(np.dot(p_k.T,Ap))  

        x_k=x_k+alpha_k*p_k

        r_k=r_k+alpha_k*Ap   

        beta_k=np.linalg.norm(r_k)**2/(r**2)
        
        p_k=-r_k+beta_k*p_k

    return n,x_k,r_k,k,bres

* Pruebe el algoritmo para resolver el sistema de ecuaciones 

$$ \mathbf{A}_1\mathbf{x}=\mathbf{b}_1$$

donde

$$ \mathbf{A}_1 = n\mathbf{I} + \mathbf{1} = 
\left[\begin{array}{llll} n      & 0      & \cdots & 0 \\
                       0      & n      & \cdots & 0 \\ 
                       \vdots & \vdots & \ddots & \vdots \\
                       0      & 0      & \cdots & n \end{array}\right]
+ \left[\begin{array}{llll} 1    & 1      & \cdots & 1 \\
                       1      & 1      & \cdots & 1 \\ 
                       \vdots & \vdots & \ddots & \vdots \\
                       1      & 1      & \cdots & 1 \end{array}\right],  \qquad
\mathbf{b}_1 = \left[\begin{array}{l} 1 \\ 1 \\ \vdots \\ 1 \end{array}\right], $$

In [11]:
#definimos la funcion que genera la matriz A de acuerdo a la instrucción anterior
def genera_A1(n):
    return np.ones((n,n))+n*np.eye(n)


$n=10$

In [12]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
x0=np.zeros(n)
b=np.ones(n)
argumentos_GCL=[x0,A,b,n,tau]
imprime_1(gradiente_conjugado_lineal,argumentos_GCL)

Dimensión de A =  10
Número de iteraciones =  1
Primeras cuatro entradas de x_k=  [0.05 0.05 0.05 0.05]
Últimas cuatro entradas de x_k=  [0.05 0.05 0.05 0.05]
Norma residual ||rk||   =  7.021666937153402e-16
Sí se cumplio el criterio de convergencia


$n=100$

In [13]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
x0=np.zeros(n)
b=np.ones(n)
argumentos_GCL=[x0,A,b,n,tau]
imprime_1(gradiente_conjugado_lineal,argumentos_GCL)

Dimensión de A =  100
Número de iteraciones =  1
Primeras cuatro entradas de x_k=  [0.005 0.005 0.005 0.005]
Últimas cuatro entradas de x_k=  [0.005 0.005 0.005 0.005]
Norma residual ||rk||   =  0.0
Sí se cumplio el criterio de convergencia


$n=1000$

In [14]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
x0=np.zeros(n)
b=np.ones(n)
argumentos_GCL=[x0,A,b,n,tau]
imprime_1(gradiente_conjugado_lineal,argumentos_GCL)

Dimensión de A =  1000
Número de iteraciones =  1
Primeras cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Últimas cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Norma residual ||rk||   =  0.0
Sí se cumplio el criterio de convergencia


* También aplique el algoritmo para resolver el sistema 

$$ \mathbf{A}_2\mathbf{x}=\mathbf{b}_2$$

donde  $\mathbf{A}_2 = [a_{ij}]$ con

$$ a_{ij} = exp\left(-0.25(i-j)^2 \right),  \qquad
\mathbf{b}_2 = \left[\begin{array}{l} 1 \\ 1 \\ \vdots \\ 1 \end{array}\right] $$


In [15]:
#definimos la funcion que genera la matriz A de acuerdo a la instrucción anterior
def genera_A2(n):
    A = np.empty([n,n], dtype=float)
    for i in range(n):
        for j in range(n):
            u=0.25*((i-j)**2)
            A[i][j]=exp(-1*u)
    return A

$n=10$

In [16]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
x0=np.zeros(n)
b=np.ones(n)
argumentos_GCL=[x0,A,b,n,tau]
imprime_1(gradiente_conjugado_lineal,argumentos_GCL)

Dimensión de A =  10
Número de iteraciones =  5
Primeras cuatro entradas de x_k=  [ 1.36909916 -1.16637682  1.60908281 -0.61339053]
Últimas cuatro entradas de x_k=  [-0.61339053  1.60908281 -1.16637682  1.36909916]
Norma residual ||rk||   =  1.1100446976946785e-11
Sí se cumplio el criterio de convergencia


$n=100$

In [17]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
x0=np.zeros(n)
b=np.ones(n)
argumentos_GCL=[x0,A,b,n,tau]
imprime_1(gradiente_conjugado_lineal,argumentos_GCL)

Dimensión de A =  100
Número de iteraciones =  99
Primeras cuatro entradas de x_k=  [ 1.44627188 -1.41632661  2.11046439 -1.42498422]
Últimas cuatro entradas de x_k=  [-1.42498188  2.11044446 -1.41633544  1.44629683]
Norma residual ||rk||   =  8.691499885210018e-05


$n=1000$

In [18]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
x0=np.zeros(n)
b=np.ones(n)
argumentos_GCL=[x0,A,b,n,tau]
imprime_1(gradiente_conjugado_lineal,argumentos_GCL)

Dimensión de A =  1000
Número de iteraciones =  262
Primeras cuatro entradas de x_k=  [ 1.44628824 -1.41635954  2.1105181  -1.42507231]
Últimas cuatro entradas de x_k=  [-1.42507231  2.1105181  -1.41635954  1.44628824]
Norma residual ||rk||   =  0.0001876613547018017
Sí se cumplio el criterio de convergencia


$\color{purple}{Ejercicio \space 2}$ <br>

In [72]:
def backtracking(x_k, f, df, p_k, rho=0.5, iter_max=500, c1=0.001, c2=0.01,alpha_ini=1):

    alpha = alpha_ini
    

    for i in range(iter_max):
        x_kp = x_k + alpha * p_k
        gp = np.dot(df(x_k), p_k)

        # Condiciones de Wolfe
        if f(x_kp) <= f(x_k) + c1 * alpha * gp and np.dot(df(x_kp), p_k) <= -c2 * gp:
            return alpha, i, True
        

        alpha = alpha * rho

    return alpha, i, False

In [89]:
def gradiente_conjugado_no_lineal(x_0,f,D_f,max_iter,tol):
    reinicios=0
    bres=False
    x_k=x_0
    f_k=f(x_k)
    g_k=D_f(x_k)
    d_k=-g_k

    for k in range(max_iter):
        g=np.linalg.norm(g_k)
        if g<tol:
            bres=True
            return x_k,g_k,k,reinicios,bres
   
        alpha_k,_,_=backtracking(x_k,f,D_f,d_k)
        x_k=x_k+alpha_k*d_k
        g_old=g_k
        g_k=D_f(x_k)

        if np.abs(np.dot(g_k,g_old))<0.2*np.dot(g_k,g_k):
            beta_k=(np.linalg.norm(g_k)**2)/(np.linalg.norm(g_old)**2) 
        else:
            beta_k=0
            reinicios=reinicios+1
            
        d_k=-g_k+beta_k*d_k

    return x_k,g_k,k,reinicios,bres

**Función cuadrática con A1**

$n=10$

In [101]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)

Dimensión de x0, n =  10
f(x_0) =  0.0
Número de iteraciones =  13
f(x_k)=  -0.24999999999789743
Primeras cuatro entradas de x_k=  [0.04999985 0.04999985 0.04999985 0.04999985]
Últimas cuatro entradas de x_k=  [0.04999985 0.04999985 0.04999985 0.04999985]
Norma del gradiente en x_k=  9.17078433484245e-06
Contador br =  13
Sí se cumplio el criterio de convergencia


$n=100$

In [103]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)

Dimensión de x0, n =  100
f(x_0) =  0.0
Número de iteraciones =  8
f(x_k)=  -0.24999999999312786
Primeras cuatro entradas de x_k=  [0.00499997 0.00499997 0.00499997 0.00499997]
Últimas cuatro entradas de x_k=  [0.00499997 0.00499997 0.00499997 0.00499997]
Norma del gradiente en x_k=  5.2430559662752785e-05
Contador br =  8
Sí se cumplio el criterio de convergencia


$n=1000$

In [104]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)

Dimensión de x0, n =  1000
f(x_0) =  0.0
Número de iteraciones =  4
f(x_k)=  -0.24999999999997527
Primeras cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Últimas cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Norma del gradiente en x_k=  9.542125853659165e-06
Contador br =  4
Sí se cumplio el criterio de convergencia


**Función cuadrática con A2**

In [107]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)

Dimensión de x0, n =  10
f(x_0) =  0.0
Número de iteraciones =  656
f(x_k)=  -1.79342078383908
Primeras cuatro entradas de x_k=  [ 1.36883518 -1.16571247  1.60818359 -0.61261601]
Últimas cuatro entradas de x_k=  [-0.61261601  1.60818359 -1.16571247  1.36883518]
Norma del gradiente en x_k=  1.9081736803415713e-05
Contador br =  478
Sí se cumplio el criterio de convergencia


In [108]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)

Dimensión de x0, n =  100
f(x_0) =  0.0
Número de iteraciones =  4999
f(x_k)=  -14.494326547188079
Primeras cuatro entradas de x_k=  [ 1.44518005 -1.41288126  2.10384349 -1.41483562]
Últimas cuatro entradas de x_k=  [-1.41483562  2.10384349 -1.41288125  1.44518005]
Norma del gradiente en x_k=  7.262791714504865e-05
Contador br =  3917


In [109]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)

Dimensión de x0, n =  1000
f(x_0) =  0.0
Número de iteraciones =  3416
f(x_k)=  -141.43695685424564
Primeras cuatro entradas de x_k=  [ 1.44287243 -1.40569481  2.09014145 -1.39405493]
Últimas cuatro entradas de x_k=  [-1.39405493  2.09014145 -1.40569481  1.44287243]
Norma del gradiente en x_k=  0.00019133642012863605
Contador br =  2664
Sí se cumplio el criterio de convergencia


**Función de Beale**

In [74]:
x0=(2,3)
n=2
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Beale,fn.D_Beale,N,tau]
imprime_2(fn.Beale,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)


Dimensión de x0, n =  2
f(x_0) =  3347.203125
Número de iteraciones =  50
f(x_k)=  6.0010502635901e-11
x_k=  [2.99998068 0.4999951 ]
Norma del gradiente en x_k=  7.033545029879855e-06
Contador br =  38
Sí se cumplio el criterio de convergencia


**Función de Himmelblau** 

In [75]:
x0=(2,4)
n=2
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Himmelblau,fn.D_Himmelblau,N,tau]
imprime_2(fn.Himmelblau,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)


Dimensión de x0, n =  2
f(x_0) =  130
Número de iteraciones =  24
f(x_k)=  8.869121880997688e-14
x_k=  [ 3.58442835 -1.84812646]
Norma del gradiente en x_k=  2.8453157735627747e-06
Contador br =  20
Sí se cumplio el criterio de convergencia


**Función de Rosenbrock**

In [81]:
x0=(-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Rosenbrock,fn.D_Rosenbrock,N,tau]
imprime_2(fn.Rosenbrock,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)


Dimensión de x0, n =  (2,)
f(x_0) =  24.199999999999996
Número de iteraciones =  77
f(x_k)=  4.0522579708831516e-12
Primeras cuatro entradas de x_k=  [1.00000201 1.00000403]
Últimas cuatro entradas de x_k=  [1.00000201 1.00000403]
Norma del gradiente en x_k=  1.999152260463426e-06
Contador br =  55
Sí se cumplio el criterio de convergencia


In [84]:
x0=(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Rosenbrock,fn.D_Rosenbrock,N,tau]
imprime_2(fn.Rosenbrock,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)


Dimensión de x0, n =  (20,)
f(x_0) =  4597.999999999999
Número de iteraciones =  694
f(x_k)=  4.612333027811992e-10
Primeras cuatro entradas de x_k=  [1.         1.         0.99999999 0.99999999]
Últimas cuatro entradas de x_k=  [0.99999537 0.99999073 0.99998141 0.99996273]
Norma del gradiente en x_k=  2.3941228825155865e-05
Contador br =  413
Sí se cumplio el criterio de convergencia


In [85]:
x0=(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Rosenbrock,fn.D_Rosenbrock,N,tau]
imprime_2(fn.Rosenbrock,n,x0,gradiente_conjugado_no_lineal,argumentos_GCNL)


Dimensión de x0, n =  (40,)
f(x_0) =  9680.0
Número de iteraciones =  1058
f(x_k)=  9.665203846222312e-10
Primeras cuatro entradas de x_k=  [1. 1. 1. 1.]
Últimas cuatro entradas de x_k=  [0.9999933  0.99998658 0.99997309 0.99994605]
Norma del gradiente en x_k=  3.6157029207687395e-05
Contador br =  624
Sí se cumplio el criterio de convergencia


$\color{purple}{Ejercicio \space 3}$ <br>

In [87]:
def gradiente_conjugado_no_lineal_HS(x_0,f,D_f,max_iter,tol):
    reinicios=0
    bres=False
    x_k=x_0
    f_k=f(x_k)
    g_k=D_f(x_k)
    d_k=-g_k

    for k in range(max_iter):
        g=np.linalg.norm(g_k)
        if g<tol:
            bres=True
            return x_k,g_k,k,reinicios,bres
   
        alpha_k,_,_=backtracking(x_k,f,D_f,d_k)
        x_k=x_k+alpha_k*d_k
        g_old=g_k
        g_k=D_f(x_k)
        y_k=g_k-g_old

        if np.abs(np.dot(g_k,g_old))<0.2*np.dot(g_k,g_k):
            beta_k=(np.dot(g_k,y_k))/(np.dot(d_k,y_k)) 
        else:
            beta_k=0
            reinicios=reinicios+1
            
        d_k=-g_k+beta_k*d_k

    return x_k,g_k,k,reinicios,bres

**Funciones cuadráticas con A1**

$n=10$

In [102]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)

Dimensión de x0, n =  10
f(x_0) =  0.0
Número de iteraciones =  13
f(x_k)=  -0.24999999999789743
Primeras cuatro entradas de x_k=  [0.04999985 0.04999985 0.04999985 0.04999985]
Últimas cuatro entradas de x_k=  [0.04999985 0.04999985 0.04999985 0.04999985]
Norma del gradiente en x_k=  9.17078433484245e-06
Contador br =  13
Sí se cumplio el criterio de convergencia


$n=100$

In [105]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)

Dimensión de x0, n =  100
f(x_0) =  0.0
Número de iteraciones =  8
f(x_k)=  -0.24999999999312786
Primeras cuatro entradas de x_k=  [0.00499997 0.00499997 0.00499997 0.00499997]
Últimas cuatro entradas de x_k=  [0.00499997 0.00499997 0.00499997 0.00499997]
Norma del gradiente en x_k=  5.2430559662752785e-05
Contador br =  8
Sí se cumplio el criterio de convergencia


$n=1000$

In [106]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)

Dimensión de x0, n =  1000
f(x_0) =  0.0
Número de iteraciones =  4
f(x_k)=  -0.24999999999997527
Primeras cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Últimas cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Norma del gradiente en x_k=  9.542125853659165e-06
Contador br =  4
Sí se cumplio el criterio de convergencia


**Funciones cuadráticas con A2**

In [110]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)

Dimensión de x0, n =  10
f(x_0) =  0.0
Número de iteraciones =  754
f(x_k)=  -1.7934207876762953
Primeras cuatro entradas de x_k=  [ 1.36886346 -1.16578522  1.60828044 -0.61270105]
Últimas cuatro entradas de x_k=  [-0.61270105  1.60828044 -1.16578522  1.36886346]
Norma del gradiente en x_k=  1.7196073436070605e-05
Contador br =  584
Sí se cumplio el criterio de convergencia


In [111]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)

Dimensión de x0, n =  100
f(x_0) =  0.0
Número de iteraciones =  4999
f(x_k)=  -14.494326214738267
Primeras cuatro entradas de x_k=  [ 1.44513466 -1.41273989  2.10357233 -1.41442268]
Últimas cuatro entradas de x_k=  [-1.41442268  2.10357233 -1.41273989  1.44513466]
Norma del gradiente en x_k=  7.578182799782151e-05
Contador br =  3769


In [112]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica y su gradiente como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
N=5000
argumentos_GCNL=[x0,cuadratica,D_cuadratica,N,tau]
imprime_2(cuadratica,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)

Dimensión de x0, n =  1000
f(x_0) =  0.0
Número de iteraciones =  3463
f(x_k)=  -141.4369563585438
Primeras cuatro entradas de x_k=  [ 1.44284074 -1.40558695  2.089946   -1.39375064]
Últimas cuatro entradas de x_k=  [-1.39375064  2.089946   -1.40558695  1.44284074]
Norma del gradiente en x_k=  0.00019135949030101222
Contador br =  2618
Sí se cumplio el criterio de convergencia


**Función de Beale**

In [90]:
x0=(2,3)
n=2
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Beale,fn.D_Beale,N,tau]
imprime_2(fn.Beale,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)


Dimensión de x0, n =  2
f(x_0) =  3347.203125
Número de iteraciones =  25
f(x_k)=  7.076693020301564e-11
x_k=  [2.99997901 0.49999469]
Norma del gradiente en x_k=  7.385377876071349e-06
Contador br =  18
Sí se cumplio el criterio de convergencia


**Función de Himmelblau** 

In [91]:
x0=(2,4)
n=2
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Himmelblau,fn.D_Himmelblau,N,tau]
imprime_2(fn.Himmelblau,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)


Dimensión de x0, n =  2
f(x_0) =  130
Número de iteraciones =  22
f(x_k)=  7.317880926980324e-13
x_k=  [ 3.5844283  -1.84812631]
Norma del gradiente en x_k=  6.824300082945526e-06
Contador br =  19
Sí se cumplio el criterio de convergencia


**Función de Rosenbrock**

In [92]:
x0=(-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Rosenbrock,fn.D_Rosenbrock,N,tau]
imprime_2(fn.Rosenbrock,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)


Dimensión de x0, n =  (2,)
f(x_0) =  24.199999999999996
Número de iteraciones =  50
f(x_k)=  1.655294559448486e-11
Primeras cuatro entradas de x_k=  [1.00000407 1.00000815]
Últimas cuatro entradas de x_k=  [1.00000407 1.00000815]
Norma del gradiente en x_k=  3.6497301985052404e-06
Contador br =  38
Sí se cumplio el criterio de convergencia


In [97]:
x0=(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Rosenbrock,fn.D_Rosenbrock,N,tau]
imprime_2(fn.Rosenbrock,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)


Dimensión de x0, n =  (20,)
f(x_0) =  4597.999999999999
Número de iteraciones =  390
f(x_k)=  2.159244904522652e-10
Primeras cuatro entradas de x_k=  [1.00000001 1.00000001 1.         1.        ]
Últimas cuatro entradas de x_k=  [0.99999686 0.99999367 0.9999873  0.99997451]
Norma del gradiente en x_k=  2.3651325537511567e-05
Contador br =  238
Sí se cumplio el criterio de convergencia


In [96]:
x0=(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
N=5000
argumentos_GCNL=[x0,fn.Rosenbrock,fn.D_Rosenbrock,N,tau]
imprime_2(fn.Rosenbrock,n,x0,gradiente_conjugado_no_lineal_HS,argumentos_GCNL)


Dimensión de x0, n =  (40,)
f(x_0) =  9680.0
Número de iteraciones =  742
f(x_k)=  1.0815875655240946e-10
Primeras cuatro entradas de x_k=  [1.         1.00000001 1.00000001 1.00000001]
Últimas cuatro entradas de x_k=  [0.99999779 0.99999556 0.99999108 0.99998212]
Norma del gradiente en x_k=  3.179932713897736e-05
Contador br =  451
Sí se cumplio el criterio de convergencia


Resumen comparativo entre gradiente conjugado no lineal usando FR y HS

    Respecto a las iteraciones:
*   B necesitó la mitad de iteraciones usando Hestenes-Stiefel en lugar de Fletcher-Reeves.
*   H necesitó dos iteraciones usando Hestenes-Stiefel en lugar de Fletcher-Reeves.
*   R necesitó un tercio menos para dos dim y al rededor la mitad de iteraciones para 20 Y 40 dim usando Hestenes-Stiefel en lugar de Fletcher-Reeves.


    Respecto a los reinicios:

*   B necesitó 10 reinicios usando Hestenes-Stiefel en lugar de Fletcher-Reeves.
*   H necesitó 1 reinicios usando Hestenes-Stiefel en lugar de 
*   R necesitó 15 reinicios menos para dos dim y al rededor de 200 reinicios para 20 Y 40 dim usando Hestenes-Stiefel en lugar de Fletcher-Reeves.


Sí hay una difencia en la cantidad de iteraciones (y de reinicios) que se obtienen usando la fórmula de HS respecto a usar FR. En general HS ahorra hasta la mitad de iteraciones en casi todos los casos. También se requieren unos pocos menos reinicios. 

